## Redes Sociales: Scraping de URLs

**Fecha de ejecución:** 22/07/2025  
**Empresas objetivo:**  
Facebook, Instagram, LinkedIn, Twitter/X, TikTok, Snapchat, Pinterest



| Empresa      | URLs únicas | Fecha      | Estado / Observaciones                                |
|--------------|------------|------------|-------------------------------------------------------|
| Facebook     |     2      | 22/07/2025 | OK – Home minimalista, muy pocos enlaces              |
| Instagram    |     0      | 22/07/2025 | No scrapeado – Home bloqueada/login requerido         |
| LinkedIn     |    151     | 22/07/2025 | OK – Muchas URLs públicas                             |
| Twitter/X    |     5      | 22/07/2025 | OK – Pocos enlaces relevantes                         |
| TikTok       |     0      | 22/07/2025 | No scrapeado – Home bloqueada/login requerido         |
| Snapchat     |    20      | 22/07/2025 | OK – Algunos enlaces accesibles                       |
| Pinterest    |     0      | 22/07/2025 | No scrapeado – Home bloqueada/login requerido         |


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd, os
import time
from urllib.parse import urljoin, urlparse
from time import sleep


In [2]:
empresas_rrss = {
    "Facebook": "https://www.facebook.com",
    "Instagram": "https://www.instagram.com",
    "LinkedIn": "https://www.linkedin.com",
    "Twitter/X": "https://twitter.com",
    "TikTok": "https://www.tiktok.com",
    "Snapchat": "https://www.snapchat.com",
    "Pinterest": "https://www.pinterest.es"
}


In [3]:


def obtener_urls(base_url, delay=1):
    """
    Descarga la página principal y extrae las URLs internas.
    Solo acepta URLs del dominio principal y subdominios.
    """
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(base_url, timeout=10, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        dominio_empresa = urlparse(base_url).netloc

        # Elimina "www." al inicio para igualar subdominios y dominio principal
        dominio_base = dominio_empresa[4:] if dominio_empresa.startswith('www.') else dominio_empresa

        urls = set()
        for link in soup.find_all('a', href=True):
            href = link['href']
            href = urljoin(base_url, href)
            dominio_href = urlparse(href).netloc
            dominio_href_base = dominio_href[4:] if dominio_href.startswith('www.') else dominio_href

            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        time.sleep(delay)  # Añade delay entre requests si scrapeas en lote
        return list(urls)

    except Exception as e:
        print(f'Error al acceder a {base_url}: {e}')
        return []


In [4]:
resultados = []
for nombre, url_base in empresas_rrss.items():
    print(f'Scraping empresa: {nombre}')
    urls_empresa = obtener_urls(url_base)
    for url in urls_empresa:
        resultados.append({'empresa': nombre, 'url': url})
    time.sleep(2)

Scraping empresa: Facebook
Scraping empresa: Instagram
Scraping empresa: LinkedIn
Scraping empresa: Twitter/X
Scraping empresa: TikTok
Scraping empresa: Snapchat
Scraping empresa: Pinterest


In [5]:
df_scrap = pd.DataFrame(resultados)
df_scrap.to_csv('../data/raw/rrss_legitimas_crudo.csv', index=False)

In [6]:
print(df_scrap['empresa'].value_counts())


empresa
LinkedIn     151
Snapchat      20
Twitter/X      5
Facebook       2
Name: count, dtype: int64


- **Empresas objetivo:** 7  
- **Empresas con resultados:** 4 (Facebook, LinkedIn, Twitter/X, Snapchat)
- **Empresas sin datos:** 3 (Instagram, TikTok, Pinterest)
- **Motivo:** Las homes de Instagram, TikTok y Pinterest requieren login o están bloqueadas para scraping anónimo.
- **Observaciones:**  
  - En RRSS es habitual que solo consigas datos en páginas realmente abiertas al público, el resto son privadas/ocultas.
  - LinkedIn es la excepción porque expone más URLs públicas.
- **Acción futura:**  
  - Si necesitas más datos de estas plataformas, habrá que probar técnicas avanzadas (selenium, autenticación, scraping indirecto, etc.), pero para un proyecto de demo/entrevista esto es suficiente si lo documentas así.
